# Bernoulli Console with Random Geometry

In [12]:
import numpy as np

rs = np.random.RandomState()
rs.seed(0)


def brownian_motion(T=1, N=100, mu=0.1, sigma=0.01, S0=20):
    dt = float(T)/N
    t = np.linspace(0, T, N)
    W = rs.standard_normal(size=N)
    W = np.cumsum(W)*np.sqrt(dt)  # standard brownian motion
    X = (mu-0.5*sigma**2)*t + sigma*W
    S = S0*np.exp(X)  # geometric brownian motion
    return S


params = dict(T=0.1, N=100, mu=0.5, sigma=0.1)

x = brownian_motion(**params)
y = brownian_motion(**params)
z = brownian_motion(**params)


In [13]:
from polymesh import PolyData, CartesianFrame, LineData
import numpy as np

frame = CartesianFrame(dim=3)

coords = np.stack([x, y, z], axis=1)
topo = np.zeros((coords.shape[0]-1, 2)).astype(int)
topo[:, 0] = np.arange(topo.shape[0])
topo[:, 1] = topo[:, 0] + 1

mesh = PolyData(coords=coords, frame=frame)
mesh['lines'] = LineData(topo=topo, frame=frame)

mesh.plot(notebook=False)


In [14]:
from sigmaepsilon.solid import Structure, LineMesh, PointData
from neumann.linalg import linspace, Vector
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from sigmaepsilon.solid.fem.cells import B2 as Beam

import numpy as np
from numpy import pi as PI

# INPUT DATA
w, h = 10., 10.  # width and height of the rectangular cross section
F = -100.  # value of the vertical load at the free end
E = 210000.0  # Young's modulus
nu = 0.3  # Poisson's ratio

# cross section
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = w * h**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)

# support at the first, load at the last node
loads = np.zeros((coords.shape[0], 6))
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
global_load_vector = Vector([0., 0, -1], frame=GlobalFrame).show()
loads[-1, :3] = global_load_vector
fixity[0, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linsolve()

# postproc
# 1) displace the mesh
dofsol = structure.nodal_dof_solution()[:, :3]


In [15]:
dofsol[-1, :3]

array([ 1.00585117e-08, -2.40120686e-09, -1.98541542e-07])